In [1]:
import pandas as pd
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np

from metrics import check_feature_distributions

In [2]:
df = pd.read_csv('data/2018.csv')

In [3]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27'],
      dtype='object')

In [5]:
def parse_info(x):
    x = x.split('-')
    return int(x[0]), int(x[1]), int(x[2])

def parse_date(df):
    res = df['FL_DATE'].apply(parse_info)
    df[['YEAR', 'MONTH', 'DAY']] = pd.DataFrame(res.tolist(), index=df.index)
    df.drop('FL_DATE', axis=1, inplace=True)

In [6]:
parse_date(df)

In [7]:
df.head()

,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,YEAR,MONTH,DAY
0,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,...,1605.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,1
1,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,...,414.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,1
2,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,...,846.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,1
3,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,...,1120.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,1
4,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,...,723.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,1


In [8]:
df['is_delayed'] = df['DEP_DELAY'] > 15

In [9]:
df.columns

Index(['OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON',
       'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'YEAR', 'MONTH', 'DAY', 'is_delayed'],
      dtype='object')

In [10]:
to_drop = [
    'OP_CARRIER',
    'OP_CARRIER_FL_NUM',
    'ORIGIN',
    'DEST',
    'DEP_TIME',
    'DEP_DELAY',
    'ARR_TIME',
    'ARR_DELAY',
    'ACTUAL_ELAPSED_TIME',
    'AIR_TIME',
    'ORIGIN',
    'DEST',
    'TAXI_OUT',
    'WHEELS_OFF',
    'WHEELS_ON',
    'TAXI_IN',
    'ARR_TIME',
    'CANCELLATION_CODE',
    'DIVERTED',
    'CARRIER_DELAY',
    'WEATHER_DELAY',
    'NAS_DELAY',
    'SECURITY_DELAY',
    'LATE_AIRCRAFT_DELAY',
    'Unnamed: 27',
    'CANCELLED'
]

In [11]:
df.drop(to_drop, axis=1, inplace=True)

In [12]:
df.columns

Index(['CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE', 'YEAR',
       'MONTH', 'DAY', 'is_delayed'],
      dtype='object')

In [13]:
df.dropna(inplace=True)

In [14]:
df.shape

(7213436, 8)

In [15]:
sample = df.sample(frac=0.1)

In [16]:
x = sample.drop('is_delayed', axis=1)
y = sample['is_delayed']

In [17]:
RANDOM_SEED = 7
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)

In [18]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [19]:
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train, y_train))
print(lr.score(x_test, y_test))

0.8243177407216699
0.8247906693037097


In [20]:
rf = RandomForestClassifier(n_estimators=40, n_jobs=-1)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [21]:
print(rf.score(x_train, y_train))
print(rf.score(x_test, y_test))

0.9984176337782706
0.8139174876619656


In [40]:
import numpy as np

def is_internally_separable(x_f, y):
    """
        Checks if an array for a feature is separable on the interior of the distribution
        x_f: feature array
        y: output
        c: class that we are identifing on the interior
    """
    m = y.mean() # useful for class imbalance
    for c in [0, 1]:
        data_c = x_f[y==c]
        upper_quartile = np.percentile(data_c, 75)
        lower_quartile = np.percentile(data_c, 25)
        iqr = upper_quartile - lower_quartile
        b_arr = (lower_quartile-1.5*iqr <= x_f) & (x_f <= upper_quartile+1.5*iqr)
        y_internal = y[b_arr]
        x_internal = x_f[b_arr]

        decision_points = [0.25, 0.5, 0.75]
        
        ym = m if c == 1 else 1 - m
        delta = 0.1 # 0.1 off the mean
        if ym > 0.85:
            delta = (1 - ym) / 3
        elif ym < 0.15:
            delta = ym / 3

        for dp in decision_points:
            end = int(len(y_internal) * dp)
            v = (y_internal[:end]==c).mean()
            if v >= ym + delta:
                return True
            elif v <= ym - delta:
                return True
    return False

def analyze_side(side, cutoff):
    m = side.mean()
    if m >= cutoff:
        return True, 1
    elif (1 - m) >= cutoff:
        return True, 0
    return False, -1
            
def is_externally_separable(x_f, y):
    """
        Returns (STATUS, DESCRIPTOR)
        where STATUS is a boolean indicating whether the externals are separable
        and DESCRIPTOR describes which class dominates both ends (only if that is the case, otherise this is -1)
    """
    upper_quartile = np.percentile(x_f, 75)
    lower_quartile = np.percentile(x_f, 25)
    
    left = y[x_f <= lower_quartile]
    right = y[x_f >= upper_quartile]
    
    
    cutoff = 0.7 # at least this much should belong to this class in interior section
    bl, dl = analyze_side(left, cutoff)
    br, dr = analyze_side(right, cutoff)
    
    # currently we shall say LR is not good only if both ends are not determined or both ends dominated by same class
    if bl and br:
        # both sides were dominated
        if dl == dr:
            # same class both sides
            return False, [dl, dr]
        else:
            # diff class on both ends
            return True, [dl, dr]
    elif bl and not br:
        # left dominated
        return True, [dl, dr]
    elif not bl and br:
        # right dominated
        return True, [dl, dr]
    else:
        # neither dominated
        return False, [dl, dr]
    
def check_feature_distributions(x, y):
    """
        Returns a dictionary showing which features are good for which model.
        Note that in general if something works for LR, it works for RF.
        We use LR over RF because it is 1) statistically/theoretically grounded, 
        2) has few assumptions, 3) directly interpretable as probabilitic, and 4) faster
    """
    result = {
        'logit': [],
        'nonlinear': [],
        'half-logit': []
    }
    for i in range(x.shape[1]):
        s, d = is_externally_separable(x[:,i], y)
        if s:
            if d[0] != -1 and d[1] != -1:
                # is externally separable
                result['logit'].append(i)
            else:
                # is half separable
                result['half-logit'].append(i)
        if is_internally_separable(x[:,i], y) or is_internally_separable(x[:,i], y):
            result['nonlinear'].append(i)
    return result

def plot_1d(x, y, f):
    x_0 = x[y==0][:,f]
    x_1 = x[y==1][:,f]
    plt.plot(x_0, np.zeros(len(x_0)), '.', alpha=0.5, color='b')
    plt.plot(x_1, np.zeros(len(x_1)) + 1, '.', alpha=0.5, color='r')

In [41]:
check_feature_distributions(x.values, y)

{'logit': [], 'nonlinear': [], 'half-logit': []}